In [ ]:
from IPython.display import display
import IPython
import pandas as pd
from pandas import Series
import numpy as np
from random import randrange
from matplotlib import pyplot
import matplotlib.pyplot as plt
import sys
sys.path.append('..')
import math
from typing import Any, List, Dict, AnyStr, Optional
from pathlib import Path
from glob import glob
from utils.dataset import MarketData, DataPreprocessor, Plotting
from utils.windowgenerator import WindowGenerator, compile_and_fit
import tensorflow as tf
import talib
%load_ext autoreload
%autoreload 2

In [68]:
eua_options_cols = ['Date', 'Aggregate Put Open Interest  (R1)', 'Aggregate Call Open Interest  (R1)', 
                    'Aggregate Open Interest  (L1)', 'OPTION OI%', 'PUT/CALL OI']
eua_options = pd.read_excel('../data/data_sheet_latest.xlsx', sheet_name='EUA option-G363')
eua_options['Date'] = pd.to_datetime(pd.to_datetime(eua_options['Date']).dt.date)
eua_options = eua_options[eua_options_cols][eua_options['Date'].dt.year >= 2018].dropna()

In [ ]:
display(eua_options.head())

In [ ]:
display(eua_options.tail())

In [ ]:
pachis_delta = pd.read_excel('../data/data_sheet_latest.xlsx', sheet_name='25Delta')
dec_cols = ["Date", "Hist Vol", "50D-Hist Vol", "50D", "25D Spread", "butterfly"]
pachis_delta_dec = pachis_delta[dec_cols]
pachis_delta_dec.columns = ["Date", "Hist Vol - 1Y", 
                        "iVol/Hist Vol Spread - Dec", 
                        "50 Delta iVol - Dec", 
                        "25Δ Risk Reversal (Call - Put) - Dec", 
                        "Butterfly - Dec"]
pachis_delta_dec['Date'] = pd.to_datetime(pachis_delta_dec['Date'])
pachis_delta_dec = pachis_delta_dec[pachis_delta_dec['Date'].dt.year > 2018] 
for col in ['Hist Vol - 1Y', 'iVol/Hist Vol Spread - Dec', '50 Delta iVol - Dec']:
    pachis_delta_dec.iloc[:, pachis_delta_dec.columns.get_loc(col)] = pachis_delta_dec[col].replace(' ', np.nan).astype(float)
pachis_delta_dec.iloc[:, pachis_delta_dec.columns.get_loc('iVol/Hist Vol Spread - Dec')] = pachis_delta_dec['iVol/Hist Vol Spread - Dec'].astype(float)
pachis_delta_dec.iloc[:, pachis_delta_dec.columns.get_loc('50 Delta iVol - Dec')] = pachis_delta_dec['50 Delta iVol - Dec'].astype(float)


In [ ]:
prompt_cols = ["Date.1", "50D.1", "25D Spread.1", "butterfly.1"]
pachis_delta_prompt = pachis_delta[prompt_cols]
pachis_delta_prompt.columns = ["Date", "50 Delta iVol - Prompt",
                               "25Δ Risk Reversal (Call - Put) - Prompt",
                               "Butterfly - Prompt"]
pachis_delta_prompt['Date'] = pd.to_datetime(pachis_delta_prompt['Date'])
pachis_delta_prompt = pachis_delta_prompt[pachis_delta_prompt['Date'].dt.year > 2018]
# First convert any string values to numeric, handling spaces
pachis_delta_prompt['50 Delta iVol - Prompt'] = pd.to_numeric(pachis_delta_prompt['50 Delta iVol - Prompt'].replace(' ', np.nan), errors='coerce')

In [161]:
option_ts = pd.read_excel('../data/data_sheet_latest.xlsx', sheet_name='Option Time series')

In [209]:
# Create prompt dataframe
cols = ['Date', 'Call_OI-Prompt', 'Put_OI-Prompt', 'Dec_OI', 'Call/Put', 'Option%']
option_ts_prompt = option_ts[cols].copy()
option_ts_prompt.columns = ['Date', 'Call_OI-Prompt', 'Put_OI-Prompt', 'Dec_OI', 'Call/Put-Prompt', 'Option%-Prompt']

# Create december dataframe
cols = ['Date', 'Call/Put.1', 'Option%.1']
option_ts_dec = option_ts[cols].copy()
option_ts_dec.columns = ['Date', 'Call/Put-Dec', 'Option%-Dec']

# Merge the two dataframes on Date
option_ts_combined = pd.merge(option_ts_prompt, option_ts_dec, on='Date', how='outer')
option_ts_combined = option_ts_combined.sort_values(by='Date', ascending=False)
option_ts_combined = option_ts_combined[:-74]

In [235]:
combined_df = eua_options.merge(pachis_delta_dec, on='Date', how='left')\
    .merge(pachis_delta_prompt, on='Date', how='left')\
    .merge(option_ts_combined, on='Date', how='left')

In [257]:
combined_df = combined_df[combined_df['Date'].dt.year>=2023][:-70]

In [ ]:
combined_df

In [ ]:
from IPython.display import display
import pandas as pd
import numpy as np
import streamlit as st
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from pathlib import Path
from pymongo import MongoClient
from datetime import datetime

from utils.dataset import MarketData, DataPreprocessor
from utils.data_processing import prepare_data

In [263]:
def load_and_preprocess_data():
    cot_df, auction_df, options_df, ta_df, fundamentals_df = MarketData.latest(Path('../data'))
    cot_df = cot_df.set_index('Date').resample('W', origin='end').mean().reset_index()
    auction_df = auction_df.set_index('Date').resample('D').mean().reset_index()

    auction_df = auction_df[7:]
    # auction_df.loc[:, 'Premium/discount-settle'] = auction_df['Premium/discount-settle'].ffill()
    auc_cols = ['Auc Price', 'Median Price', 'Cover Ratio', 'Spot Value', 
                'Auction Spot Diff', 'Median Spot Diff', 'Premium/discount-settle']
    auction_df.loc[:, auc_cols] = auction_df[auc_cols].ffill()

    merged_df = DataPreprocessor.engineer_auction_features(auction_df)
    return merged_df, options_df

In [ ]:
merged_df, options_df = load_and_preprocess_data()

In [ ]:

options_df

In [27]:
from IPython.display import display
import pandas as pd
import numpy as np
import streamlit as st
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from pathlib import Path
from pymongo import MongoClient
from datetime import datetime
import sys
sys.path.append("../")
from utils.dataset import MarketData, DataPreprocessor
from utils.data_processing import prepare_data, reverse_normalize
from utils.plotting import (display_performance_metrics, display_trade_log, plot_equity_curve, 
                            plot_model_results_with_trades, plot_recent_predictions, plot_ensemble_statistics, plot_ensemble_predictions_realtime)
from utils.model_utils import create_model, train_model, generate_model_predictions, train_ensemble_models
from utils.mongodb_utils import get_stored_predictions, setup_mongodb_connection, save_recent_predictions
from utils.backtesting import backtest_model_with_metrics

In [ ]:
cot_df, auction_df, options_df, ta_df, fundamentals_df = MarketData.latest(Path('../data'))
cot_df = cot_df.set_index('Date').resample('W', origin='end').mean().reset_index()
auction_df = auction_df.set_index('Date').resample('D').mean().reset_index()

auction_df = auction_df[7:]
# auction_df.loc[:, 'Premium/discount-settle'] = auction_df['Premium/discount-settle'].ffill()
auc_cols = ['Auc Price', 'Median Price', 'Cover Ratio', 'Spot Value', 
            'Auction Spot Diff', 'Median Spot Diff', 'Premium/discount-settle']
auction_df.loc[:, auc_cols] = auction_df[auc_cols].ffill()

merged_df = DataPreprocessor.engineer_auction_features(auction_df)

In [ ]:
options_df.shape

In [80]:
dup_dates = options_df['Date'].value_counts()[:10].index

In [ ]:
dup_dates

In [ ]:
options_df[options_df['Date'].isin(dup_dates)]

In [ ]:
auc_df = merged_df[['Date', 'Auc Price']].copy()
options_df = options_df.merge(auc_df, how='inner')
options_df = options_df.bfill()

In [47]:
options_df = options_df.set_index('Date')

In [ ]:
options_df.shape

In [ ]:
options_df['Date'].value_counts()

In [17]:
def prepare_data_and_train_model(merged_df):
    train_df, test_df, val_df, preprocessor = prepare_data(merged_df)
    num_features = len(test_df.columns)
    OUT_STEPS = 7
    model = create_model(num_features, OUT_STEPS)
    history = train_model(model, train_df, val_df, test_df, preprocessor)
    predictions_df, recent_preds, trend = generate_model_predictions(model, test_df)
    
    return model, preprocessor, test_df, predictions_df, recent_preds, trend

In [ ]:
train_df, test_df, val_df, preprocessor = prepare_data(options_df)

In [6]:
num_features = len(test_df.columns)
OUT_STEPS = 7
model = create_model(num_features, OUT_STEPS)

In [ ]:
history = train_model(model, train_df, val_df, test_df, preprocessor)

In [ ]:
predictions_df, recent_preds, trend = generate_model_predictions(model, test_df)

In [ ]:
trade_log_df, performance_metrics, balance_history_df = backtest_model_with_metrics(
                    model,
                    test_df,
                    7,
                    7,
    10000,
    4 / 100.0,
    2 / 100.0,
    position_size_fraction=100 / 100.0,
    risk_free_rate=1 / 100.0,
    preprocessor=preprocessor
)



In [23]:
input_width = 7
out_steps= 7
initial_balance = 10000
take_profit = 4 / 100.0
stop_loss = 2 / 100.0
position_size_fraction =100 / 100.0
risk_free_rate=1 / 100.0

In [24]:
features = test_df.columns
num_features = len(features)
predictions = []

# Initialize balance and trade log
balance = initial_balance
balance_history = []
trade_log = []

# For performance metrics
returns = []
equity_curve = [initial_balance]

In [ ]:
# Generate predictions in steps of 'out_steps'
for idx, i in enumerate(range(input_width, len(test_df) - out_steps + 1, out_steps)):
    try:
        inputs = test_df[i - input_width:i].values
        inputs_reshaped = inputs.reshape((1, input_width, num_features))
        preds = model.predict(inputs_reshaped)
        predictions.append(preds[0])
    except Exception as e:
        print(f"Prediction error at index {i}: {e}")
        break

In [28]:
# Create DataFrame for predictions
predictions = np.concatenate(predictions, axis=0)
pred_indices = test_df.index[input_width:input_width + len(predictions)]
predictions_df = pd.DataFrame(predictions, columns=features, index=pred_indices)
# Reverse normalize predictions and test data
predictions_df = reverse_normalize(predictions_df, preprocessor.train_mean['Auc Price'], preprocessor.train_std['Auc Price'])
test_df_denormalized = reverse_normalize(test_df.copy(), preprocessor.train_mean['Auc Price'], preprocessor.train_std['Auc Price'])


In [ ]:
for idx, i in enumerate(range(input_width, len(test_df) - out_steps + 1, out_steps)):
    entry_index = predictions_df.index[idx * out_steps]
    entry_price = test_df_denormalized.loc[entry_index, 'Auc Price']
    prev_index = entry_index - pd.Timedelta(days=1)
    prev_price = test_df_denormalized.loc[prev_index, 'Auc Price'] if prev_index in test_df_denormalized.index else entry_price
    print(f"IDX: {idx} | i: {i}")
    print(f"Entry Index: {entry_index} | Entry Price: {entry_price} | ")
    print(f"Prev Index: {prev_index} | Prev Price: {prev_price}")
    print()
    if idx == 2: break